In [1]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: C:\Users\mubee\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import cv2

In [3]:
import time

In [22]:
object_detector = cv2.createBackgroundSubtractorMOG2()# Object detection from Stable camera
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))

start_time = float(0)
video = cv2.VideoCapture("Car.mp4")
total_distance=150 # meters

while (video.isOpened()):
    ret, frame = video.read()
    
    if ret == True:
        # Extracting Region of Interest for Calibraion  
        height, width, _ = frame.shape
        roi = frame[150: 250,150: 350]

        #Filtering Noise from frame
        mask = object_detector.apply(roi)
        _, mask = cv2.threshold(mask, 254, 255, cv2.THRESH_BINARY)
        
        #Removing Car trails
        opening = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=10)
        close = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel, iterations=10)

        #Setting Boundary Box
        boundaries, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        
        #NonMaxima Supperision
        boundary = None
        if boundaries:
            boundary=boundaries[0]
        xmin=-1
        ymax = 9999
        for i in boundaries:
            boundary_area = cv2.contourArea(i)
            if boundary_area >300:
                xi, yi, wi, hi = cv2.boundingRect(i)
                if xi > xmin and yi < ymax:
                    xmin = xi
                    ymax = yi
                    boundary = i
                    
        x, y, w, h = cv2.boundingRect(boundary)
        
        #Calculating speed
        
        if x == 150:
            start_time = time.time()
        
        time_elapsed = (time.time()-start_time)
        diatance_covered = total_distance - x
       
        
        if time_elapsed != 0:
            speed=diatance_covered/time_elapsed
            speed = round(speed, 2)
        if speed<0:
            speed = 0
        
        #Setting Output
        cv2.putText(roi, str(speed) +"m/s" , (x, y - 10), cv2.FONT_HERSHEY_PLAIN, 1, (0, 255, 0), 1)
        cv2.rectangle(roi, (x, y), (x + w, y + h), (0, 0, 255), 2)
        cv2.imshow("Frame",frame)
        #cv2.imshow("ROI",roi)
        #cv2.imshow("Mask", mask)
                #cv2.drawContours(frame,[i],-1, (0,0,255), 2)          
          
    else:
        break    

    #press ESC to break
    key=cv2.waitKey(30)
    if key == 27:
        break
 

video.release()
cv2.destroyAllWindows()

